
# Задача - анализ вероятностной модели данных
Цель - построение регрессионной модели

Загрузка данных

In [1]:
import arviz as az
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pymc3 as pm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl
WARN: Could not locate executable gfortran
WARN: Could not locate executable f95
WARN: Could not locate executable g95
WARN: Could not locate executable efort
WARN: Could not locate executable efc
WARN: Could not locate executable flang
WARN: don't know how to compile Fortran code on platform 'nt'


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data = pd.read_csv('./data/flats.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/flats.csv'

In [ ]:
data.sample(10)

In [ ]:
data.describe()

Исходя из анализа данных сделаем
- Unnamed: 0, на мой взгляд нумерациия в таблице - удалим ее из датасета
- переименуем колонки 
- удалим из рассмотрения данные которые неправдободобын (например жилая площадт равна 0 или 1)
В датасете содержится следующие признаки: цена, общая площадь, жилая площадь, колисетво комнат и площадь кухни!

Согласно центральной предельной теореме - все распределения сводятся в нормальному распределению! Поэтому примем распределения параметров как нормльное.

Таблица не содержитт пропусков

In [ ]:
data.isnull().values.any()

In [ ]:
data.rename({'price_doc': 'price', 'full_sq': 'area', 'life_sq': 'life','num_room': 'room','kitch_sq': 'kitchen', }, axis=1, inplace=True)

In [ ]:
data.sample(10)

In [ ]:
data.describe()

In [ ]:
data = data[["price","area","life","room","kitchen"]]
data['price_log'] = np.log(data['price'])
data.describe()

Промашстабируем параметры,( у меня рассчитывается вероятностная модель рассчитывается долго) , так как мы применяем регрессияяб регрессия линеная модель и масштабирование не влияет на линеную модель

In [ ]:
# scalerFeat = StandardScaler()
# dataScaler = scalerFeat.fit_transform(data)

dateFrame = data[["price_log","area","life","room","kitchen"]]
dateFrame.describe()

In [ ]:
# from theano import tensor as T
logistic_model = pm.Model()

with pm.Model() as logistic_model:
    pm.glm.GLM.from_formula('price_log ~ area + life + room + kitchen ', dateFrame, family = pm.glm.families.Normal())
    trace = pm.sample(1000)

Вероятностая модель

In [ ]:
logistic_model

In [ ]:
pm.traceplot(trace, figsize = (12, 12))

In [ ]:
pm.plot_posterior(trace, figsize = (12, 10), kind="hist")

In [ ]:
pm.summary(trace).round(2)

## Сравнение с логистической регрессией

In [ ]:
# Приведение типов датафрейма 
convert_dict = {'price_log': float, 'area': float, 'life': float, 'room': float, 'kitchen': float, }
dateFrame = dateFrame.astype(convert_dict)
dateFrame

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(dateFrame[['area', 'life',  'room', 'kitchen']], dateFrame['price_log'])

In [ ]:
lr.intercept_

In [ ]:
lr.coef_


In [ ]:
pd.DataFrame(lr.coef_, index=['area', 'life',  'room', 'kitchen'])

Вывод:
 - была построена вероятностная модель с помощью Байесовского метода, проверена гипотеза нормальнео распределение параметров
 - построена линейная реграссия
 - метод Байесовского с нормальным распределением параметров совпадает с коэффициентами линейнной регрессии
 
 
## В качестве интересса построим модель Байесовского с гипотезой  распределения параметров по распределению Стьюдента
 

In [ ]:
dataBi = pd.read_csv('./data/flats.csv')
dataBi.rename({'price_doc': 'price', 'full_sq': 'area', 'life_sq': 'life','num_room': 'room','kitch_sq': 'kitchen', }, axis=1, inplace=True)

dataBi['price_log'] = np.log(dataBi['price'])
dataBi.describe()

In [ ]:
import pymc3 as pm
bi_model = pm.Model()

with pm.Model() as bi_model:
    pm.glm.GLM.from_formula('price_log ~ area + life + room + kitchen ', dataBi, family=pm.glm.families.StudentT())
    trace = pm.sample(1000)

In [ ]:
pm.traceplot(trace, figsize = (12, 12))

In [ ]:
pm.plot_posterior(trace, figsize = (12, 10), kind="hist");

In [ ]:
pm.summary(trace).round(2)